<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Случайное блуждание в 2D

### Examples - Chemistry

<section class="post-meta">
By Håkon Ånes, Jonas Tjemsland, Andreas Krogen and Jon Andreas Støvneng
</section>
Last edited: March 22nd 2018 
___

## Вступление

Линейные *полимеры* - это молекулы, которые образуют длинные цепочки основных звеньев, называемых *мономерами*, и цепи могут достигать $10^{5}$ мономеров. Реальные линейные полимерные цепи можно увидеть на рис. 1. Ученые хотят знать, сколько различных конфигураций может принимать цепь из $n$-мономеров, а также как далеко друг от друга обычно находятся конечные точки полимера, предполагая, что каждая конфигурация одинаково вероятна [[1]](#rsc). Эти вопросы переводятся в вопросы о *произвольных блужданиях*, которые мы рассмотрим в этом блокноте.

<a title="By Yurko (Own work) [CC BY-SA 3.0 (http://creativecommons.org/licenses/by-sa/3.0)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3ASingle_Polymer_Chains_AFM.jpg"><img width="175" alt="Single Polymer Chains AFM" src="https://upload.wikimedia.org/wikipedia/commons/0/03/Single_Polymer_Chains_AFM.jpg"/></a>

**Рисунок 1:** Внешний вид реальных линейных полимерных цепей, записанных с помощью атомно-силового микроскопа, на поверхности в жидкой среде. Длина контура цепи для этого полимера составляет ~204 нм; толщина ~0.4 нм. Взято из [[2]](rsc#).

Произвольные блуждания - это путь по решетке, который не посещает один и тот же агент более одного раза. Несмотря на простоту, на многие вопросы, касающиеся этой модели, трудно ответить строгим математическим способом. В частности, возникают вопросы о том, как далеко обычно проходит ходок сделавший $n$ шагов. Отец и сын П. С. Хеммер и С. Хеммер опубликовали в 1984 году краткую статью с понятным названием *Среднее случайное блуждание, избегающее себя, по квадратной решетке длится 71 шаг* [[3]](#rsc). В следующем мы всего несколькими строками кода покажем, как они пришли к своему выводу.

## Произвольные блуждания с избежанием самопересечения

На квадратной решетке в двух измерениях ходок может попасть в ловушку. Примером этого являются прогулки, показанные на рис. 2. На рисунке показаны все возможные произвольные блуждания, состоящие из семи, восьми или девяти шагов.

<img width="400" alt="Self-trapped walks with seven, eight or nine steps." src="images/self_trapped_walks.png" />

**Рисунок 2:** Произвольные блуждания с семью, восемью или девятью шагами. Адаптировано из [[3]](#rsc).

Скажем, что на каждом шаге дается равная вероятность перехода на соседние сайты, которые ранее не посещались. Для низких $n$ вероятность того, что $t(n)$ произойдет самозахват после $n$ шагов в течение 1-9 шагов, равна:

$$
t(1) = ... = t(6) = 0; \:\: t(7) = \frac{2}{729}; \\
t(8) = \frac{5}{2187}; \:\: t(9) = \frac{31}{6561}.
$$

Эти распределения $t(n)$ могут быть вычислены путем выполнения достаточно большого числа самоустраняющихся случайных блужданий по решетке. Для каждого шага, на какой сайт в конечном итоге перемещается ходок, определяется случайной выборкой.

## Реализация функции случайного ходока

Мы сохраняем два массива. Количество шагов каждой прогулки записывается в массив `walk_lengths`, в то время как расстояние $R$ между ходоком и его начальной точкой записывается в массив `displacements`.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import progressbar
import warnings
warnings.filterwarnings('ignore')

In [ ]:
newparams = {'font.size': 20, 'lines.linewidth': 2, 'figure.figsize': (14, 7) }
plt.rcParams.update(newparams)

In [ ]:
def random_walk2D(m, walks):
    """Perform self-avoiding random walks on an m-by-m (square) lattice.
    
    Parameters:
        m:       int. Dimension of lattice.
        walks:   int. Number of walks to perform.
    
    Returns:
        walk_lengths:  list. List of lengths of each walk.
        displacements: list. List of displacement from starting point of each walk.
    """
    
    walk_lengths = [0]*walks
    displacements = [0]*walks
    # Create a progress bar for the for loop
    bar = progressbar.ProgressBar()
    for i in bar(range(walks)):
        # Create an m-by-m array (lattice) with all elements set to zero
        a = np.zeros([m, m])
        # Index the array using integer numbers. Walker starts in the
        # middle of the array.
        x = int(m/2)
        y = int(m/2)
        steps = 0
        while (x > 0) and (x < m-1) and (y > 0) and (y < m-1):
            # Check if trapped. If not, make a random move.
            a[x][y] = 1
            if a[x-1][y] and a[x+1][y] and a[x][y-1] and a[x][y+1]:
                walk_lengths[i] = steps
                # Calculate displacement from starting point
                displacements[i] = np.sqrt((x - int(m/2))**2 + (y - int(m/2))**2)
                break
            # Random sampling
            r = np.random.randint(1, 5)
            if (r == 1) and (not a[x+1][y]):
                x += 1
                steps += 1
            elif (r == 2) and (not a[x-1][y]):
                x -= 1
                steps += 1
            elif (r == 3) and (not a[x][y+1]):
                y += 1
                steps += 1
            elif (r == 4) and (not a[x][y-1]):
                y -= 1
                steps += 1
    return walk_lengths, displacements

Достаточно большая квадратная решетка необходима для того, чтобы случайный ходок не сошел с нее. Но более крупная решетка приведет к увеличению времени вычислений. Предыдущие прогоны показали, что наибольшие смещения обычно находятся в радиусе 100 точек решетки. Таким образом, будет достаточно определить решетку $m \times m$, $200\times 200$. Если ходок *должен* отклониться от этой решетки, он получит длину нулевых шагов. Как показано на рис. 2, наименьшее количество шагов, на которые может попасть случайный ходок, - это семь. Чтобы проверить, достаточно ли велика наша решетка, мы можем, таким образом, проверить кратчайший путь и увеличить размер решетки, если кратчайший путь равен нулю.

## Запуск проходов

В своей статье Хеммеры выполнили 60 000 проходов и построили график распределения вероятности $t(n)$ для самозахвата, который произойдет после $n$ шагов. Давайте сделаем то же самое и сравним наши результаты с их.

In [ ]:
m = 200
walks = 60000
walk_lengths, displacements = random_walk2D(m, walks)
np.min(walk_lengths)

In [ ]:
# Plot the probability distribution
plt.hist(walk_lengths, bins=np.max(walk_lengths), color='k', label='Probabilities');
plt.xlabel('$n$');
plt.ylabel('$t(n)$');
plt.xlim([0, 300]);

# Plot the gross features of the distribution in an approximate formula
n = np.linspace(0, 300, 1000)
plt.plot(n, 0.37e-2*(n - 6)**(3/5)*np.exp(-n/40), label='Approximation');
plt.legend();

Общие характеристики нашего распределения могут быть выражены приближенными формулами, например [[3]](#rsc)

$$
t(n) \propto (n-6)^{3/5}e^{-n/40}.
$$

Добавление коэффициента масштабирования $0.37\cdot10^{-2}$ соответствует приближению к нашим данным. Приближение показано на графике выше.

Теперь мы можем из массивов `walk_lenghts` и `displacements` рассчитать среднюю длину ходьбы со стандартным отклонением, наиболее вероятную длину, максимальную длину, среднее смещение от начальной точки и максимальное смещение от начальной точки.

In [ ]:
# Calculate parameters
mean_len = np.mean(walk_lengths)
std_len = np.std(walk_lengths)/np.sqrt(walks)
most_prob_len = np.bincount(walk_lengths).argmax()
max_len = np.max(walk_lengths)

mean_dis = np.mean(displacements)
std_dis = np.std(displacements)/np.sqrt(walks)
max_dis = np.max(displacements)

print('An average self-avoiding random walk on the square lattice lasts %.1f ± %.1f steps (70.7 ± 0.2).' 
      % (mean_len, std_len))
print('Most probable length: %i (33).' % most_prob_len)
print('Maximum length: ', (max_len))
print('Average displacement R = %.2f ± %.2f (11.87 ± 0.05).' % (mean_dis, std_dis))
print('Maximum displacement: %.2f.' % (max_dis))

Результаты Хеммерса приведены в приведенных выше параграфах. Наши результаты довольно близки к результатам, приведенным в статье.

### Дополнительные вопросы
* Как выглядит распределение вероятностей для смещения?
* Какие еще свойства можно изучить?
* Как самоизбегающий случайный ходок ведет себя на гексагональной решетке?
* Или в трех измерениях?

<a id="rsc"></a>
### References

[1] Neal, M and Slade, G. [The self-avoiding walk](https://books.google.no/books?hl=no&lr=&id=JsoFCAAAQBAJ&oi=fnd&pg=PR11&dq=gordon+slade+self-avoiding+walk&ots=ov8bIRDCbL&sig=TzPF_P_V1cB_-6apq2tPPhPYZlw&redir_esc=y#v=onepage&q=gordon%20slade%20self-avoiding%20walk&f=false). Springer Science & Business Media, 2013. 

[2] Roiter, Y and Minko, S. [AFM Single Molecule Experiments at the Solid-Liquid Interface: In Situ Conformation of Adsorbed Flexible Polyelectrolyte Chains](http://pubs.acs.org/doi/abs/10.1021/ja0558239), Journal of the American Chemical Society, vol. 127, iss. 45, pp. 15688-15689 (2005).

[3] Hemmer, S and Hemmer, P C. [An average self-avoiding random walk on the square lattice lasts 71 steps](http://scitation.aip.org/content/aip/journal/jcp/81/1/10.1063/1.447349). J. Chem. Phys **81** (1), 1 July 1984.